# Jobs command

### Introduction

The `job` command can manage jobs in the current group.

* list: list all jobs
* get: get a job information by id
* submit: submit a job from file or schedule
* rerun: rerun a job by id
* wait: wait a job until timeout or finish
* cancel: cancel a job by id
* logs: get job logs by id
* list_artifacts: list all artifacts of a job 
* download_artifacts: download job artifects

## Setup PrimeHub Python SDK


In [ ]:
from primehub import PrimeHub, PrimeHubConfig
ph = PrimeHub(PrimeHubConfig())

if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
else:
    print("PrimeHub Python SDK couldn't get the group information, follow the get-started.ipynb to complete it")

## Help documentation

In [ ]:
help(ph.jobs)

## Examples

### Get detail information of jobs

In [ ]:
# List all jobs or with page number
all_jobs = ph.jobs.list()
job_page_1 = ph.jobs.list(page = 1)

print("Number of all jobs:", len(all_jobs))
print("Number of jobs at page 1:", len(job_page_1))

In [ ]:
# Get detail information by id
ph.jobs.get(id='job-202107290141-h54kz1')

### Submit jobs and get the logs

In [ ]:
# Submit a job
config = {
    "instanceType": "cpu-1",
    "image": "base-notebook",
    "displayName": "short-job",
    "command": "echo \"test1\"\necho \"test2\"",
}

short_job = ph.jobs.submit(config)
print(short_job)

In [ ]:
# Wait the job to be done
print('[ Waiting ]')
ph.jobs.wait(short_job['id'])
print('[ Job Done ]')

# Get logs
logs = ph.jobs.logs(short_job['id'])
print('[ Job Logs ]')
for l in logs:
    print(l)

In [ ]:
# Get the job logs in stream mode
config = {
    'instanceType': 'cpu-1',
    'image': 'base-notebook',
    'displayName': 'long-job',
    'command': 'for i in {1..10}; do echo $i; sleep 1; done',
}
long_job  = ph.jobs.submit(config)

##
## TODO: need a way to wait unitl the job is running before get the logs
##
ph.jobs.wait(long_job['id'], timeout=10)

logs = ph.jobs.logs(long_job['id'], follow=True)
print('[ Job Logs ]')
for l in logs:
    print(l)

### Rerun and cancel the job

In [ ]:
# Rerun the job
rerun_job = ph.jobs.rerun(long_job['id'])
print('Rerun Job:')
print(rerun_job)

##
## TODO: need a way to wait unitl the job is running before get the logs
##
import time
ph.jobs.wait(rerun_job['id'], timeout=10)

# Cancel the job
cancel_job = ph.jobs.cancel(rerun_job['id'])
print('Cancel Job:')
print(cancel_job)

### Job Artifacts
more information: https://docs.primehub.io/docs/job-artifact-feature

In [ ]:
# Submit a job that creates job artifacts
config = {
    'instanceType': 'cpu-1',
    'image': 'base-notebook',
    'displayName': 'job-with-artifacts',
    'command': 'mkdir -p /home/jovyan/artifacts/sub\n echo \"hello\" > /home/jovyan/artifacts/test.txt\n echo \"hello\" > /home/jovyan/artifacts/sub/test.txt',
}
job_with_artifacts = ph.jobs.submit(config)

# Wait the job to be done
ph.jobs.wait(job_with_artifacts['id'])

In [ ]:
# List artifacts of a job
ph.jobs.list_artifacts(job_with_artifacts['id'])

In [ ]:
# Download artifacts
ph.jobs.download_artifacts(job_with_artifacts['id'], 'test.txt', '/tmp')    
!cat /tmp/test.txt

In [ ]:
##
## TODO: add download recursive
##